# 自定义嵌入
LangChain 已集成众多[第三方嵌入模型](/docs/integrations/text_embedding/)。本指南将演示如何创建自定义嵌入类，以应对内置模型未覆盖的场景。嵌入技术在自然语言处理应用中至关重要，它能将文本转化为算法可理解的数值形式，从而支持相似性搜索、文本分类和聚类等多样化应用场景。
通过标准的[Embeddings](https://python.langchain.com/api_reference/core/embeddings/langchain_core.embeddings.embeddings.Embeddings.html)接口实现嵌入功能，可使您的嵌入模型兼容现有`LangChain`抽象层（例如作为驱动[VectorStore](https://python.langchain.com/api_reference/core/vectorstores/langchain_core.vectorstores.base.VectorStore.html)的嵌入引擎，或通过[CacheBackedEmbeddings](/docs/how_to/caching_embeddings/)实现缓存功能）。
## 接口
当前LangChain中的`Embeddings`抽象层专为处理文本数据而设计。在此实现中，输入可以是单个字符串或字符串列表，输出则是数值数组（向量）的列表，其中每个向量代表将输入文本嵌入到某个n维空间中。
您的自定义嵌入必须实现以下方法：
| 方法/属性                  | 描述                                                                 | 必填/选填          ||---------------------------------|----------------------------------------------------------------------------|-------------------|| `embed_documents(texts)`        | 为字符串列表生成嵌入向量                                | 必需          || `embed_query(text)`             | 为单个文本查询生成嵌入向量。                            | 必填          || `aembed_documents(texts)`       | 异步为字符串列表生成嵌入向量。                 | 可选          || `aembed_query(text)`            | 异步为单个文本查询生成嵌入向量。             | 可选          |
这些方法确保您的嵌入模型能够无缝集成到LangChain框架中，同时提供同步和异步功能以实现可扩展性和性能优化。

:::note`Embeddings` 目前未实现 [Runnable](/docs/concepts/runnables/) 接口，同时也**不是** pydantic 的 `BaseModel` 实例。:::
### 嵌入查询与文档
`embed_query` 和 `embed_documents` 方法是必需的。这两个方法都用于操作对字符串输入的处理。访问 `Document.page_content` 属性的操作已得到处理由于历史遗留原因，向量存储库使用了该嵌入模型。
`embed_query` 接收单个字符串作为输入，并返回一个浮点数列表形式的单个嵌入向量。如果您的模型针对嵌入查询和底层文档有不同的模式，您可以实现此方法来处理该问题。
`embed_documents` 接收一个字符串列表作为输入，并返回一个嵌入向量列表（即由浮点数组成的列表的列表）。
:::note`embed_documents` 接收的是纯文本列表，而非 LangChain 的 `Document` 对象列表。此方法的名称在 LangChain 的未来版本中可能会发生变化。好的，请提供需要翻译的英文文本，我会按照标准Markdown格式将其翻译成中文并直接输出内容。例如：

**示例输入：**
```markdown
# Introduction  
This is a **sample** text with _markdown_ formatting.  
- Item 1  
- Item 2  
```

**示例输出：**
# 简介  
这是一段带有 **Markdown** 格式的 _示例_ 文本。  
- 项目1  
- 项目2  

请直接提供您的英文内容，我会立即处理。

## 实现
例如，我们将实现一个返回常量向量的简单嵌入模型。该模型仅用于演示目的。

In [1]:
from typing import List

from langchain_core.embeddings import Embeddings


class ParrotLinkEmbeddings(Embeddings):
    """ParrotLink embedding model integration.

    # TODO: Populate with relevant params.
    Key init args — completion params:
        model: str
            Name of ParrotLink model to use.

    See full list of supported init args and their descriptions in the params section.

    # TODO: Replace with relevant init params.
    Instantiate:
        .. code-block:: python

            from langchain_parrot_link import ParrotLinkEmbeddings

            embed = ParrotLinkEmbeddings(
                model="...",
                # api_key="...",
                # other params...
            )

    Embed single text:
        .. code-block:: python

            input_text = "The meaning of life is 42"
            embed.embed_query(input_text)

        .. code-block:: python

            # TODO: Example output.

    # TODO: Delete if token-level streaming isn't supported.
    Embed multiple text:
        .. code-block:: python

             input_texts = ["Document 1...", "Document 2..."]
            embed.embed_documents(input_texts)

        .. code-block:: python

            # TODO: Example output.

    # TODO: Delete if native async isn't supported.
    Async:
        .. code-block:: python

            await embed.aembed_query(input_text)

            # multiple:
            # await embed.aembed_documents(input_texts)

        .. code-block:: python

            # TODO: Example output.

    """

    def __init__(self, model: str):
        self.model = model

    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        """Embed search docs."""
        return [[0.5, 0.6, 0.7] for _ in texts]

    def embed_query(self, text: str) -> List[float]:
        """Embed query text."""
        return self.embed_documents([text])[0]

    # optional: add custom async implementations here
    # you can also delete these, and the base class will
    # use the default implementation, which calls the sync
    # version in an async executor:

    # async def aembed_documents(self, texts: List[str]) -> List[List[float]]:
    #     """Asynchronous Embed search docs."""
    #     ...

    # async def aembed_query(self, text: str) -> List[float]:
    #     """Asynchronous Embed query text."""
    #     ...

### 让我们来测试一下

In [2]:
embeddings = ParrotLinkEmbeddings("test-model")
print(embeddings.embed_documents(["Hello", "world"]))
print(embeddings.embed_query("Hello"))

[[0.5, 0.6, 0.7], [0.5, 0.6, 0.7]]
[0.5, 0.6, 0.7]


## 贡献指南
我们欢迎向LangChain代码库贡献嵌入模型。
如果您希望为新的提供商贡献一个嵌入模型（例如，使用一组新的依赖项或SDK），我们建议您将实现发布在单独的 `langchain-*` 集成包中。这将使您能够妥善管理依赖关系并为您的包进行版本控制。请参考我们的[贡献指南](/docs/contributing/how_to/integrations/)以了解此过程的详细说明。